In [1]:
import logging
import os.path
import time
from collections import OrderedDict
import sys

import numpy as np
import torch.nn.functional as F
from torch import optim

from braindecode.models.deep4 import Deep4Net
from braindecode.models.eegnet import EEGNetv4
from braindecode.datasets.bcic_iv_2a import BCICompetition4Set2A
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, \
    RuntimeMonitor
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.datautil.splitters import split_into_two_sets
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.torch_ext.util import set_random_seeds, np_to_var
from braindecode.mne_ext.signalproc import mne_apply
from braindecode.datautil.signalproc import (bandpass_cnt,
                                             exponential_running_standardize)
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

log = logging.getLogger(__name__)

<p>Here's the description from the paper</p>
<img src="DeepConvNet.png" style="width: 700px; float:left;">

<p>Here's the description from the paper</p>
<img src="DeepConvNetDetail.png" style="width: 700px; float:left;">

In [2]:
def run_exp(data_folder, subject_id, low_cut_hz, model, cuda):
    ival = [-500, 4000]
    max_epochs = 200
    max_increase_epochs = 16
    batch_size = 60
    high_cut_hz = 38
    factor_new = 1e-3
    init_block_size = 1000
    valid_set_fraction = 0.2

    #Loading data 
    
    train_filename = 'A{:02d}T.gdf'.format(subject_id)
    test_filename = 'A{:02d}E.gdf'.format(subject_id)
    train_filepath = os.path.join(data_folder, train_filename)
    test_filepath = os.path.join(data_folder, test_filename)
    train_label_filepath = train_filepath.replace('.gdf', '.mat')
    test_label_filepath = test_filepath.replace('.gdf', '.mat')

    train_loader = BCICompetition4Set2A(
        train_filepath, labels_filename=train_label_filepath)
    test_loader = BCICompetition4Set2A(
        test_filepath, labels_filename=test_label_filepath)
    train_cnt = train_loader.load()
    test_cnt = test_loader.load()

    # Preprocessing

    train_cnt = train_cnt.drop_channels(['STI 014', 'EOG-left',
                                         'EOG-central', 'EOG-right'])
    assert len(train_cnt.ch_names) == 22
    # lets convert to millvolt for numerical stability of next operations
    train_cnt = mne_apply(lambda a: a * 1e6, train_cnt)
    train_cnt = mne_apply(
        lambda a: bandpass_cnt(a, low_cut_hz, high_cut_hz, train_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), train_cnt)
    train_cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=factor_new,
                                                  init_block_size=init_block_size,
                                                  eps=1e-4).T,
        train_cnt)

    test_cnt = test_cnt.drop_channels(['STI 014', 'EOG-left',
                                       'EOG-central', 'EOG-right'])
    assert len(test_cnt.ch_names) == 22
    test_cnt = mne_apply(lambda a: a * 1e6, test_cnt)
    test_cnt = mne_apply(
        lambda a: bandpass_cnt(a, low_cut_hz, high_cut_hz, test_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), test_cnt)
    test_cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=factor_new,
                                                  init_block_size=init_block_size,
                                                  eps=1e-4).T,
        test_cnt)

    marker_def = OrderedDict([('Left Hand', [1]), ('Right Hand', [2],),
                              ('Foot', [3]), ('Tongue', [4])])

    train_set = create_signal_target_from_raw_mne(train_cnt, marker_def, ival)
    test_set = create_signal_target_from_raw_mne(test_cnt, marker_def, ival)

    train_set, valid_set = split_into_two_sets(
        train_set, first_set_fraction=1-valid_set_fraction)

    set_random_seeds(seed=20190706, cuda=cuda)
    
    #Run experiment
    n_classes = 4
    n_chans = int(train_set.X.shape[1])
    input_time_length = train_set.X.shape[2]
    if model == 'shallow':
        model = ShallowFBCSPNet(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    elif model == 'deep':
        model = Deep4Net(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    elif model == 'EEGNet':
        model = EEGNetv4(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    if cuda:
        model.cuda()
    #log.info("Model: \n{:s}".format(str(model)))

    optimizer = optim.Adam(model.parameters())

    iterator = BalancedBatchSizeIterator(batch_size=batch_size)

    stop_criterion = Or([MaxEpochs(max_epochs)])
                         #,NoDecrease('valid_misclass', max_increase_epochs)])

    monitors = [LossMonitor(), MisclassMonitor(), RuntimeMonitor()]

    model_constraint = MaxNormDefaultConstraint()

    exp = Experiment(model, train_set, valid_set, test_set, iterator=iterator,
                     loss_function=F.nll_loss, optimizer=optimizer,
                     model_constraint=model_constraint,
                     monitors=monitors,
                     stop_criterion=stop_criterion,
                     remember_best_column='valid_misclass',
                     run_after_early_stop=True, cuda=cuda)
    exp.run()
    return exp

In [3]:
if __name__ == '__main__':
    logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                        level=logging.DEBUG, stream=sys.stdout)
    # Should contain both .gdf files and .mat-labelfiles from competition
    data_folder = './data/BCICIV_2a_gdf/'
    subject_id = 1 # 1-9
    low_cut_hz = 4 # 0 or 4
    model = 'EEGNet' #'shallow' or 'deep'
    cuda = False
    exp = run_exp(data_folder, subject_id, low_cut_hz, model, cuda)
    log.info("Last 10 epochs")
    log.info("\n" + str(exp.epochs_df.iloc[-10:]))

Extracting EDF parameters from ./data/BCICIV_2a_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


/anaconda3/lib/python3.6/site-packages/mne/io/edf/edf.py:1083: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')


Reading 0 ... 672527  =      0.000 ...  2690.108 secs...
Extracting EDF parameters from ./data/BCICIV_2a_gdf/A01E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


/anaconda3/lib/python3.6/site-packages/mne/io/edf/edf.py:1083: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')


2019-02-10 12:01:09,954 INFO : Trial per class:
Counter({'Tongue': 72, 'Foot': 72, 'Right Hand': 72, 'Left Hand': 72})
2019-02-10 12:01:10,286 INFO : Trial per class:
Counter({'Left Hand': 72, 'Right Hand': 72, 'Foot': 72, 'Tongue': 72})
2019-02-10 12:01:10,723 INFO : Run until first stop...


...

2019-02-10 13:16:38,065 INFO : Epoch 166
2019-02-10 13:16:38,066 INFO : train_loss                0.12020
2019-02-10 13:16:38,066 INFO : valid_loss                0.83831
2019-02-10 13:16:38,067 INFO : test_loss                 0.62493
2019-02-10 13:16:38,067 INFO : train_misclass            0.00435
2019-02-10 13:16:38,068 INFO : valid_misclass            0.29310
2019-02-10 13:16:38,068 INFO : test_misclass             0.27778
2019-02-10 13:16:38,069 INFO : runtime                   3.86266
2019-02-10 13:16:38,069 INFO : 
2019-02-10 13:16:40,301 INFO : Time only for training updates: 2.23s
2019-02-10 13:16:42,040 INFO : Epoch 167
2019-02-10 13:16:42,040 INFO : train_loss                0.13576
2019-02-10 13:16:42,041 INFO : valid_loss                0.89157
2019-02-10 13:16:42,041 INFO : test_loss                 0.64348
2019-02-10 13:16:42,041 INFO : train_misclass            0.01304
2019-02-10 13:16:42,042 INFO : valid_misclass            0.32759
2019-02-10 13:16:42,043 INFO : test_m

2019-02-10 13:17:27,074 INFO : valid_misclass            0.29310
2019-02-10 13:17:27,075 INFO : test_misclass             0.28819
2019-02-10 13:17:27,075 INFO : runtime                   4.13572
2019-02-10 13:17:27,076 INFO : 
2019-02-10 13:17:29,115 INFO : Time only for training updates: 2.04s
2019-02-10 13:17:30,621 INFO : Epoch 180
2019-02-10 13:17:30,621 INFO : train_loss                0.10227
2019-02-10 13:17:30,622 INFO : valid_loss                0.82495
2019-02-10 13:17:30,622 INFO : test_loss                 0.64284
2019-02-10 13:17:30,623 INFO : train_misclass            0.00000
2019-02-10 13:17:30,623 INFO : valid_misclass            0.27586
2019-02-10 13:17:30,623 INFO : test_misclass             0.29167
2019-02-10 13:17:30,624 INFO : runtime                   3.94167
2019-02-10 13:17:30,624 INFO : 
2019-02-10 13:17:32,566 INFO : Time only for training updates: 1.94s
2019-02-10 13:17:34,062 INFO : Epoch 181
2019-02-10 13:17:34,063 INFO : train_loss                0.10867
2

2019-02-10 13:18:16,126 INFO : test_loss                 0.62059
2019-02-10 13:18:16,127 INFO : train_misclass            0.00435
2019-02-10 13:18:16,127 INFO : valid_misclass            0.25862
2019-02-10 13:18:16,128 INFO : test_misclass             0.27431
2019-02-10 13:18:16,128 INFO : runtime                   3.57401
2019-02-10 13:18:16,129 INFO : 
2019-02-10 13:18:16,131 INFO : New best valid_misclass: 0.258621
2019-02-10 13:18:16,132 INFO : 
2019-02-10 13:18:18,207 INFO : Time only for training updates: 2.07s
2019-02-10 13:18:19,879 INFO : Epoch 194
2019-02-10 13:18:19,880 INFO : train_loss                0.08709
2019-02-10 13:18:19,880 INFO : valid_loss                0.75461
2019-02-10 13:18:19,880 INFO : test_loss                 0.60790
2019-02-10 13:18:19,881 INFO : train_misclass            0.00000
2019-02-10 13:18:19,881 INFO : valid_misclass            0.27586
2019-02-10 13:18:19,882 INFO : test_misclass             0.27431
2019-02-10 13:18:19,882 INFO : runtime        

2019-02-10 13:19:02,275 INFO : Time only for training updates: 2.43s
2019-02-10 13:19:03,919 INFO : Epoch 199
2019-02-10 13:19:03,919 INFO : train_loss                0.14970
2019-02-10 13:19:03,920 INFO : valid_loss                0.32826
2019-02-10 13:19:03,920 INFO : test_loss                 0.59155
2019-02-10 13:19:03,920 INFO : train_misclass            0.02778
2019-02-10 13:19:03,921 INFO : valid_misclass            0.12069
2019-02-10 13:19:03,921 INFO : test_misclass             0.26042
2019-02-10 13:19:03,922 INFO : runtime                   4.08796
2019-02-10 13:19:03,922 INFO : 
2019-02-10 13:19:06,388 INFO : Time only for training updates: 2.46s
2019-02-10 13:19:08,038 INFO : Epoch 200
2019-02-10 13:19:08,038 INFO : train_loss                0.13982
2019-02-10 13:19:08,039 INFO : valid_loss                0.27691
2019-02-10 13:19:08,039 INFO : test_loss                 0.54904
2019-02-10 13:19:08,040 INFO : train_misclass            0.02083
2019-02-10 13:19:08,040 INFO : va

KeyboardInterrupt: 

In [4]:
log.info("Last 10 epochs")
log.info("\n" + str(exp.epochs_df.iloc[-10:]))

2019-02-10 13:19:36,633 INFO : Last 10 epochs


NameError: name 'exp' is not defined